# Скрипт для создания и заполнения БД

## Парсинг JSON и формирование sql скрипта

In [1]:
import pandas as pd
import numpy as np

In [15]:
import json

### Скрип создания базы и переменные БД

In [2]:
DATABASE_OWNER = 'postgres' # Юзер БД (админ сервера)
ENCODING = 'UTF8' # Выбранная кодировка должна соответствовать кодировке сервера (на всякий случай)
LOCALE = 'ru_RU'

DATABASE_NAME = 'glemapanida'
SCHEMA_NAME = 'router'
TABLE_NAME = 'City'

In [3]:
create_db_q = f"""CREATE DATABASE {DATABASE_NAME} WITH
OWNER "{DATABASE_OWNER}"
ENCODING '{ENCODING}'
LC_COLLATE = '{LOCALE}.{ENCODING}'
LC_CTYPE = '{LOCALE}.{ENCODING}'
TEMPLATE template0;

\c {DATABASE_NAME};
set client_encoding TO '{ENCODING}';
"""

create_schema_table_q = f"""
CREATE SCHEMA {SCHEMA_NAME}
    CREATE TABLE {TABLE_NAME} (
        id SERIAL PRIMARY KEY,
        name TEXT NOT NULL,
        code TEXT NOT NULL,
        code_yandex TEXT NOT NULL,
        latitude NUMERIC(10, 7) NOT NULL,
        longitude NUMERIC(10, 7) NOT NULL
    );

CREATE INDEX name_code_idx ON {SCHEMA_NAME}.{TABLE_NAME} ((lower(name)), code);
"""

### Города, координаты, международные коды

In [4]:
city_data = pd.read_excel('data/cities.xlsx').drop_duplicates().reset_index(drop=True)
city_data['name'] = city_data['name'].str.strip()
city_data['code'] = city_data['code'].str.strip()

In [5]:
city_data.head()

,name,latitude,longitude,code
0,Адыгейск,44.878372,39.190172,ADY
1,Майкоп,44.609827,40.100653,MAY
2,Горно-Алтайск,51.958268,85.960296,GOA
3,Алейск,52.492091,82.779415,ALY
4,Барнаул,53.348115,83.779836,BAX


In [6]:
city_data.isna().any()

name         False
latitude     False
longitude    False
code         False
dtype: bool

In [7]:
city_data.dtypes

name          object
latitude     float64
longitude    float64
code          object
dtype: object

In [8]:
city_data.shape

(1119, 4)

In [13]:
city_data = city_data.drop_duplicates(subset=['name'], keep=False).reset_index(drop=True)

In [14]:
city_data.shape

(1078, 4)

Потери при удалении одноимённых городов: 41 наименование

### Коды яндекса

In [23]:
with open('data/allYandex.json', 'r', encoding=f'{ENCODING}') as f:
    yac = json.load(f)

In [67]:
ya_city_code = {
    'name': [],
    'code_yandex': []
}

for country in yac['countries']:
    if country['title'] == 'Россия':
        for region in country['regions']:
            for settlement in region['settlements']:
                if settlement['title'] in city_data.name.unique().tolist():
                    if settlement['title'] in ya_city_code['name']:
                        idx = ya_city_code['name'].index(settlement['title'])
                        del ya_city_code['name'][idx]
                        del ya_city_code['code_yandex'][idx]
                    else:
                        ya_city_code['name'].append(settlement['title'])
                        ya_city_code['code_yandex'].append(settlement['codes']['yandex_code'])

In [68]:
ya_city_code_df = pd.DataFrame(ya_city_code)

In [69]:
ya_city_code_df.name.isin(city_data.name.unique().tolist()).sum()

966

Потери при удалении... какой-то хуйни: 112 наименований

In [76]:
city_data[~city_data.name.isin(ya_city_code_df.name.unique().tolist())].name.unique()

array(['Свободный', 'Знаменск', 'Строитель', 'Сельцо', 'Киржач',
       'Костерево', 'Муром', 'Покров', 'Новохоперск', 'Россошь',
       'Иваново', 'Комсомольск', 'Родники', 'Шуя', 'Карабулак', 'Саянск',
       'Черемхово', 'Майский', 'Чегем', 'Гусев', 'Пионерский',
       'Светлогорск', 'Светлый', 'Белоусово', 'Ермолино', 'Жуков',
       'Вилючинск', 'Елизово', 'Сосновка', 'Буй', 'Мантурово', 'Нерехта',
       'Горячий Ключ', 'Артемовск', 'Зеленогорск', 'Сосновоборск',
       'Щучье', 'Дмитриев', 'Отрадное', 'Пикалево', 'Сосновый Бор',
       'Козьмодемьянск', 'Ардатов', 'Дубна', 'Ивантеевка', 'Истра',
       'Королев', 'Куровское', 'Ликино-Дулево', 'Озеры', 'Пущино',
       'Щелково', 'Полярный', 'Бор', 'Горбатов', 'Городец', 'Заволжье',
       'Лысково', 'Павлово', 'Первомайск', 'Семенов', 'Боровичи',
       'Купино', 'Дмитровск', 'Городище', 'Каменка', 'Верещагино',
       'Горнозаводск', 'Гремячинск', 'Усолье', 'Артем', 'Находка',
       'Остров', 'Зверево', 'Рыбное', 'Северо-Кури

In [79]:
city_data = pd.merge(
    city_data,
    ya_city_code_df,
    how='inner',
    on='name',
    validate='1:1'
)

city_data.head()

,name,latitude,longitude,code,code_yandex
0,Адыгейск,44.878372,39.190172,ADY,c11005
1,Майкоп,44.609827,40.100653,MAY,c1093
2,Горно-Алтайск,51.958268,85.960296,GOA,c11319
3,Алейск,52.492091,82.779415,ALY,c11234
4,Барнаул,53.348115,83.779836,BAX,c197


### Скрипт вставки данных

In [8]:
def extract_info(city):
    city_name = city['name']
    city_lat = city.latitude
    city_lon = city.longitude
    city_code = city.code
    city_code_yandex = city.code_yandex
    
    return f"""INSERT INTO {SCHEMA_NAME}.{TABLE_NAME}(name, code, code_yandex, latitude, longitude) VALUES('{city_name}', '{city_code}', '{city_code_yandex}', {city_lat}, {city_lon});
"""

In [9]:
insert_into_table_q = f"""
""" + ''.join(city_data.apply(extract_info, axis=1).values.tolist())

In [10]:
query_str = create_db_q + create_schema_table_q + insert_into_table_q

In [11]:
with open('create_db.sql', 'w', encoding=f'{ENCODING}') as f:
    f.write(query_str)

In [12]:
with open('create_db.sql', 'r', encoding=f'{ENCODING}') as f:
    print(f.read())

CREATE DATABASE glemapanida WITH
OWNER "postgres"
ENCODING 'UTF8'
LC_COLLATE = 'ru_RU.UTF8'
LC_CTYPE = 'ru_RU.UTF8'
TEMPLATE template0;

\c glemapanida;
set client_encoding TO 'UTF8';

CREATE SCHEMA router
    CREATE TABLE City (
        id SERIAL PRIMARY KEY,
        name TEXT NOT NULL,
        code TEXT NOT NULL,
        latitude NUMERIC(10, 7) NOT NULL,
        longitude NUMERIC(10, 7) NOT NULL
    );

CREATE INDEX name_code_idx ON router.City ((lower(name)), code);

INSERT INTO router.City(name, code, latitude, longitude) VALUES('Адыгейск', 'ADY', 44.8783715, 39.190172);
INSERT INTO router.City(name, code, latitude, longitude) VALUES('Майкоп', 'MAY', 44.6098268, 40.1006527);
INSERT INTO router.City(name, code, latitude, longitude) VALUES('Горно-Алтайск', 'GOA', 51.9582681, 85.9602957);
INSERT INTO router.City(name, code, latitude, longitude) VALUES('Алейск', 'ALY', 52.4920914, 82.7794145);
INSERT INTO router.City(name, code, latitude, longitude) VALUES('Барнаул', 'BAX', 53.3481145,

## Команды

__1. Создание и заполнение БД__: `psql -U postgres -q -b -f create_db.sql`\
__2. Проверка кодировки сервера__: `show server_encoding;`

__3. DB checks__
```SQL
select *
from router.city
where lower(name) like 'мо%';

select name, count(name) as dup_cnt
from router.city c 
group by name
having count(name) > 1;

select * from router.city where name = 'Ялта';

select count(*), (select count(*) from router.city)
from (
	select *
	from router.city
	group by id, name, latitude, longitude
) as dedup;
```